<a href="https://colab.research.google.com/github/MaferRb/Practica/blob/main/BORRADOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv'
df = pd.read_csv(url)

In [ ]:
df

,Unnamed: 0,NIT,y,Ingresos de actividades ordinarias,Costo de ventas,Ganancia bruta,Otros ingresos,Gastos de ventas,Gastos de administración,Otros gastos,...,Prima de emisión,Acciones propias en cartera,Inversión suplementaria al capital asignado,Otras participaciones en el patrimonio,Superavit por revaluación,Otras reservas,Ganancias acumuladas,Patrimonio total,Total de patrimonio y pasivos,_merge
0,0,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
1,1,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
2,2,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
3,3,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
4,4,800095068.0,0,9514013.0,7554030.0,1959983.0,1665.0,436294.0,537606.0,383833.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3919158.0,4291158.0,8115039.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,12768,901202882.0,0,0.0,0.0,0.0,0.0,100.0,170.0,83.0,...,0.0,0.0,NaN,0.0,0.0,0.0,-353.0,647.0,917.0,both
12769,12769,901209458.0,0,0.0,NaN,0.0,0.0,NaN,62450.0,374.0,...,NaN,NaN,NaN,NaN,NaN,NaN,-63454.0,-62454.0,1361958.0,both
12770,12770,901216979.0,0,258644.0,219700.0,38944.0,0.0,NaN,17653.0,1053.0,...,NaN,NaN,NaN,NaN,NaN,NaN,11791.0,36791.0,219013.0,both
12771,12771,901218142.0,0,150529.0,NaN,150529.0,0.0,NaN,52028.0,NaN,...,1509391.0,NaN,NaN,3436642.0,NaN,7042654.0,-15662.0,13871762.0,14788446.0,both


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from scipy import stats

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score, roc_auc_score

In [ ]:
def Xy(df,target):
    """
    Split the data in X,y to ML implementations
    """
    X = df.loc[ : , df.columns != target]
    y = df[target].astype('int')
    return X,y

In [ ]:
def grid_lr(X_train, y_train):
    model = LogisticRegression(random_state=666, max_iter=1000)
    class_weight =  [{0:0.05, 1:0.95}, {0:0.1, 1:0.9}, {0:0.2, 1:0.8}]
    solvers = ['liblinear']
    penalty = ['l2','l1']
    c_values = [ 10, 1.0, 0.1, 0.01, 0.001, ]
    grid = dict(solver=solvers,penalty=penalty,C=c_values, class_weight= class_weight)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,scoring='f1',error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return  grid_result.best_estimator_

In [ ]:
url = "https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv"
df  = pd.read_csv(url)

In [ ]:
features = ['Activos corrientes totales', 'Pasivos corrientes totales' , 'Ganancia bruta', 'Patrimonio total', 'Total pasivos', 'Total de patrimonio y pasivos', 'y', 'Propiedades planta y equipo']
df = df[features]
df.dropna(inplace=True)
df['razon_corriente']=  df['Activos corrientes totales'] / df['Pasivos corrientes totales']
df['rentabilidad_patrimonio'] = df['Ganancia bruta'] / df['Patrimonio total']
df['endeudamiento'] = df['Total pasivos'] / df[ 'Total de patrimonio y pasivos']
df['short_run_apalancamiento'] = df['Pasivos corrientes totales'] / df['Patrimonio total']

In [ ]:
featuresy = ['razon_corriente', 'rentabilidad_patrimonio', 'endeudamiento' ,'short_run_apalancamiento' ,'y']
df = df[featuresy]
df.dropna(inplace=True)

In [ ]:
import pandas as pd
import numpy as np

def clean(df):
    index = ~df.isin([ np.inf, -np.inf]).any(axis=1)
    return df[index].astype(np.float64)

In [ ]:
df = clean(df)
X,y = Xy(df, 'y')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2507
           1       0.00      0.00      0.00        35

    accuracy                           0.98      2542
   macro avg       0.49      0.50      0.49      2542
weighted avg       0.97      0.98      0.97      2542



In [ ]:
! pip install imbalanced-learn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X_res, y_res = rus.fit_resample(X, y)

In [ ]:
y_res.value_counts()

0    174
1    174
Name: y, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, shuffle = True, random_state = 666, stratify=y_res)
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.80      0.23      0.36        35
           1       0.55      0.94      0.69        35

    accuracy                           0.59        70
   macro avg       0.68      0.59      0.53        70
weighted avg       0.68      0.59      0.53        70



In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # esta es la libería del clasificador de DT

In [ ]:
url = "https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv"

In [ ]:
df  = pd.read_csv(url)

In [ ]:
predictors = ['Activos corrientes totales', 'Pasivos corrientes totales' , 'Ganancia bruta', 'Patrimonio total', 'Total pasivos', 'Total de patrimonio y pasivos', 'y']
df = df[predictors]
df.dropna(inplace=True)
df['razon_corriente']=  df['Activos corrientes totales'] / df['Pasivos corrientes totales']
df['rentabilidad_patrimonio'] = df['Ganancia bruta'] / df['Patrimonio total']
df['endeudamiento'] = df['Total pasivos'] / df[ 'Total de patrimonio y pasivos']
df['short_run_apalancamiento'] = df['Pasivos corrientes totales'] / df['Patrimonio total']

In [ ]:
predictors = ['razon_corriente', 'rentabilidad_patrimonio', 'endeudamiento' ,'short_run_apalancamiento' ,'y']
df = df[predictors]
df.dropna(inplace=True)

In [ ]:
import pandas as pd
import numpy as np

def clean(df):
    index = ~df.isin([ np.inf, -np.inf]).any(axis=1)
    return df[index].astype(np.float64)

df = clean(df)

In [ ]:
X = df[predictors]
y = df['y']

In [ ]:
y.value_counts()

0.0    12535
1.0      174
Name: y, dtype: int64

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
clf.fit(X, y) # ajuste o entre de X y y

DecisionTreeClassifier()

In [ ]:
df['yhat'] = clf.predict(X) # keep in df the predictions

In [ ]:
df

,razon_corriente,rentabilidad_patrimonio,endeudamiento,short_run_apalancamiento,y,yhat
0,4.170808,0.231989,0.199422,0.069972,0.0,0.0
1,4.170808,0.231989,0.199422,0.069972,0.0,0.0
2,4.170808,0.231989,0.199422,0.069972,0.0,0.0
3,4.170808,0.231989,0.199422,0.069972,0.0,0.0
4,2.228661,0.456749,0.471209,0.707628,0.0,0.0
...,...,...,...,...,...,...
12767,69.733333,0.044617,0.014340,0.014549,0.0,0.0
12768,3.396296,0.000000,0.294438,0.417311,0.0,0.0
12769,0.942019,-0.000000,1.045856,-22.807378,0.0,0.0
12770,1.201902,1.058520,0.832015,4.952896,0.0,0.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['y'], df['yhat']))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     12535
         1.0       1.00      1.00      1.00       174

    accuracy                           1.00     12709
   macro avg       1.00      1.00      1.00     12709
weighted avg       1.00      1.00      1.00     12709



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2) # split database keeping 20% to test

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test) # 20% que no ha visto
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2510
         1.0       1.00      1.00      1.00        32

    accuracy                           1.00      2542
   macro avg       1.00      1.00      1.00      2542
weighted avg       1.00      1.00      1.00      2542



In [ ]:
X = df[predictors]
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2) # split database keeping 20% to test
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test) # 20% que no ha visto
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2507
         1.0       1.00      1.00      1.00        35

    accuracy                           1.00      2542
   macro avg       1.00      1.00      1.00      2542
weighted avg       1.00      1.00      1.00      2542

